In [2]:
import pandas as pd

df_ecuador = pd.read_csv('../DATASET/CLEAN/df_ecuador.csv')

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Veamos las features y realizamos one-hot encoding en 'oponente'
features = ['partidos_previos', 'porcentaje_ganado']
X = pd.get_dummies(df_ecuador, columns=['oponente'], drop_first=True)
X = X[features + [col for col in X.columns if 'oponente_' in col]]

# Convertir 'resultado' a numero, transforma las etiquetas categóricas en valores numéricos, asignando un número único a cada etiqueta.
le = LabelEncoder()
y = le.fit_transform(df_ecuador['resultado'])

# Separar datos en entrenamiento y prueba
#test_size=0.2: Indica que el 20% de los datos se asignarán al conjunto de prueba (X_test, y_test) y el 80% restante al conjunto de entrenamiento (X_train, y_train)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
display(X_train.head())
display(X_test.head())

(463, 65) (116, 65) (463,) (116,)


,partidos_previos,porcentaje_ganado,oponente_Armenia,oponente_Australia,oponente_Belarus,oponente_Bolivia,oponente_Brazil,oponente_Bulgaria,oponente_Canada,oponente_Cape Verde,...,oponente_Spain,oponente_Sweden,oponente_Switzerland,oponente_Trinidad and Tobago,oponente_Turkey,oponente_Uganda,oponente_United States,oponente_Uruguay,oponente_Venezuela,oponente_Zambia
226,15,0.133333,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
391,39,0.282051,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
195,0,0.500000,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
87,0,0.500000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
260,17,0.117647,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


,partidos_previos,porcentaje_ganado,oponente_Armenia,oponente_Australia,oponente_Belarus,oponente_Bolivia,oponente_Brazil,oponente_Bulgaria,oponente_Canada,oponente_Cape Verde,...,oponente_Spain,oponente_Sweden,oponente_Switzerland,oponente_Trinidad and Tobago,oponente_Turkey,oponente_Uganda,oponente_United States,oponente_Uruguay,oponente_Venezuela,oponente_Zambia
86,0,0.500000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,0,0.500000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
110,14,0.500000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
357,38,0.105263,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
204,22,0.409091,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
from sklearn.linear_model import LogisticRegression

# Inicializa y entrena el modelo de Regresión Logistica
logreg = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)

# Hacer predicciones con el set de pruebas
y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:, 1] # Probabilidad de ganar

In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report

# Evaluación del módulo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)

# Recalcula y_pred_proba para obtener probabilidades para todas las clases
y_pred_proba = logreg.predict_proba(X_test)
roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')


print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")

# Reporte
print("\nReporte:\n", classification_report(y_test, y_pred))



Accuracy: 0.5172
Precision: 0.3338
Recall: 0.4415
F1-score: 0.3755
ROC AUC Score: 0.6136

Reporte:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        30
           1       0.44      0.44      0.44        36
           2       0.56      0.88      0.68        50

    accuracy                           0.52       116
   macro avg       0.33      0.44      0.38       116
weighted avg       0.38      0.52      0.43       116



In [39]:
# Análisis
print("\n Análisis:")
print(f"Accuracy: El modelo tiene una exactitud del {accuracy:.4f}, lo que significa que predice correctamente un poco más de la mitad de las veces. Esto puede indicar que el modelo no es muy preciso y hay margen de mejora.")
print(f"Precision: Un valor de {precision:.4f} sugiere que cuando el modelo predice gana, solo acierta alrededor del 33.38% de las veces. Esto indica una alta tasa de falsos positivos.")
print(f"Recall: Con un valor de {recall:.4f}, el modelo está capturando alrededor del 44.15% de los casos positivos reales. Esto implica que hay una cantidad significativa de falsos negativos.")
print(f"F1-score: Un valor de {f1:.4f} indica un rendimiento moderado, pero con margen de mejora en la precisión y la recuperación.")
print(f"ROC AUC Score: Un valor de {roc_auc:.4f} sugiere un rendimiento aceptable, mejor que una clasificación aleatoria (0.5), pero aún con espacio para mejorar. Un valor más cercano a 1 indicaría un mejor rendimiento.")



 Análisis:
Accuracy: El modelo tiene una exactitud del 0.5172, lo que significa que predice correctamente un poco más de la mitad de las veces. Esto puede indicar que el modelo no es muy preciso y hay margen de mejora.
Precision: Un valor de 0.3338 sugiere que cuando el modelo predice gana, solo acierta alrededor del 33.38% de las veces. Esto indica una alta tasa de falsos positivos.
Recall: Con un valor de 0.4415, el modelo está capturando alrededor del 44.15% de los casos positivos reales. Esto implica que hay una cantidad significativa de falsos negativos.
F1-score: Un valor de 0.3755 indica un rendimiento moderado, pero con margen de mejora en la precisión y la recuperación.
ROC AUC Score: Un valor de 0.6136 sugiere un rendimiento aceptable, mejor que una clasificación aleatoria (0.5), pero aún con espacio para mejorar. Un valor más cercano a 1 indicaría un mejor rendimiento.


In [12]:
# Creamos un df_prediction con todos los oponentes únicos
oponentes = sorted(df_ecuador['oponente'].unique())
df_prediction = pd.DataFrame({'oponente': oponentes})
df_prediction = pd.get_dummies(df_prediction, columns=['oponente'], drop_first=True)

# Añadir features e inicializar en 0
features = ['partidos_previos', 'porcentaje_ganado']
for feature in features:
    df_prediction[feature] = 0

# Calcular partidos_previos and porcentaje_ganado
for oponente in oponentes:
    partidos_pasados = df_ecuador[df_ecuador['oponente'] == oponente]
    partido_anterior = len(partidos_pasados)
    ganados = len(partidos_pasados[partidos_pasados['resultado'] == 'gana'])
    porcentaje_ganado = ganados / partido_anterior if partido_anterior > 0 else 0.5
    
    # Actualizar df_prediction
    columna_oponente = 'oponente_' + oponente
    if columna_oponente in df_prediction.columns:
        df_prediction.loc[df_prediction[columna_oponente] == 1, 'partidos_previos'] = partido_anterior
        df_prediction.loc[df_prediction[columna_oponente] == 1, 'porcentaje_ganado'] = porcentaje_ganado

# Busca columnas faltantes en la columna df_prediction y lo llena con 0 para que sea neutro
columnas_faltantes = set(X_train.columns) - set(df_prediction.columns)
for col in columnas_faltantes:
    df_prediction[col] = 0
    
# Reordena columnas para que se empate con X_train
df_prediction = df_prediction[X_train.columns]

# Analiza el df_prediction DataFrame
print(df_prediction.shape)
display(df_prediction.head())

(64, 65)


C:\Users\Test\AppData\Local\Temp\ipykernel_14836\4236755396.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6666666666666666' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_prediction.loc[df_prediction[columna_oponente] == 1, 'porcentaje_ganado'] = porcentaje_ganado


,partidos_previos,porcentaje_ganado,oponente_Armenia,oponente_Australia,oponente_Belarus,oponente_Bolivia,oponente_Brazil,oponente_Bulgaria,oponente_Canada,oponente_Cape Verde,...,oponente_Spain,oponente_Sweden,oponente_Switzerland,oponente_Trinidad and Tobago,oponente_Turkey,oponente_Uganda,oponente_United States,oponente_Uruguay,oponente_Venezuela,oponente_Zambia
0,0,0.000000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,1.000000,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,3,0.666667,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1,0.000000,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,38,0.526316,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [14]:
df_prediction

,partidos_previos,porcentaje_ganado,oponente_Armenia,oponente_Australia,oponente_Belarus,oponente_Bolivia,oponente_Brazil,oponente_Bulgaria,oponente_Canada,oponente_Cape Verde,...,oponente_Spain,oponente_Sweden,oponente_Switzerland,oponente_Trinidad and Tobago,oponente_Turkey,oponente_Uganda,oponente_United States,oponente_Uruguay,oponente_Venezuela,oponente_Zambia
0,0,0.000000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,1.000000,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,3,0.666667,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1,0.000000,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,38,0.526316,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,1,0.000000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
60,13,0.384615,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
61,50,0.160000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
62,33,0.484848,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [16]:
# Crea la columna probabilidad de ganar para todos los posibles partidos
df_prediction['probabilidad_de_ganar'] = logreg.predict_proba(df_prediction)[:, 1]

# Ordenar por probabilidad de ganar
df_prediction = df_prediction.sort_values(by='probabilidad_de_ganar', ascending=False)

# Mostrar el resultado
display(df_prediction)

,partidos_previos,porcentaje_ganado,oponente_Armenia,oponente_Australia,oponente_Belarus,oponente_Bolivia,oponente_Brazil,oponente_Bulgaria,oponente_Canada,oponente_Cape Verde,...,oponente_Sweden,oponente_Switzerland,oponente_Trinidad and Tobago,oponente_Turkey,oponente_Uganda,oponente_United States,oponente_Uruguay,oponente_Venezuela,oponente_Zambia,probabilidad_de_ganar
30,5,0.800000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0.660033
63,2,1.000000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,0.637703
57,2,1.000000,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,0.637703
24,8,0.625000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0.635977
4,38,0.526316,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0.609710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,2,0.000000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0.157864
22,2,0.000000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0.156218
31,4,0.000000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0.147009
36,3,0.000000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0.144989


In [18]:
#Se imprime esta lista para ver el nombre del oponente y poder reemplazar
#abajo en el campo de oponente a predecir

oponentes

['Argentina',
 'Armenia',
 'Australia',
 'Belarus',
 'Bolivia',
 'Brazil',
 'Bulgaria',
 'Canada',
 'Cape Verde',
 'Catalonia',
 'Chile',
 'Colombia',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'El Salvador',
 'England',
 'Estonia',
 'Finland',
 'France',
 'Galicia',
 'German DR',
 'Germany',
 'Greece',
 'Guatemala',
 'Haiti',
 'Honduras',
 'Iran',
 'Iraq',
 'Italy',
 'Jamaica',
 'Japan',
 'Jordan',
 'Kuwait',
 'Lebanon',
 'Mexico',
 'Netherlands',
 'Nigeria',
 'North Macedonia',
 'Oman',
 'Panama',
 'Paraguay',
 'Peru',
 'Poland',
 'Portugal',
 'Qatar',
 'Republic of Ireland',
 'Romania',
 'Saudi Arabia',
 'Scotland',
 'Senegal',
 'Serbia',
 'South Africa',
 'South Korea',
 'Spain',
 'Sweden',
 'Switzerland',
 'Trinidad and Tobago',
 'Turkey',
 'Uganda',
 'United States',
 'Uruguay',
 'Venezuela',
 'Zambia']

In [33]:
#Se puede poner cualquir país de la lista
prediccion_oponente = df_prediction[df_prediction['oponente_Chile'] == 1]

In [35]:
probabilidad_de_ganar_oponente = prediccion_oponente['probabilidad_de_ganar'].values[0]
print(f"Probabilidad de que Ecuador gane al oponente: {probabilidad_de_ganar_oponente:.4f}")

Probabilidad de que Ecuador gane al oponente: 0.3626
